# scoring 

In [29]:
def scoring_function_project2(dataset_file):
    import pandas as pd
    import numpy as np
    import pickle
    from sklearn.model_selection import train_test_split, cross_val_score
    from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report
    from sklearn.model_selection import GridSearchCV
    import shap
    from sklearn.ensemble import GradientBoostingClassifier
    import h2o
    from h2o.estimators import H2OGradientBoostingEstimator
    import lightgbm as lgb
    from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
    from h2o.grid.grid_search import H2OGridSearch
    import lightgbm as lgb
    import category_encoders as ce
    from sklearn.preprocessing import StandardScaler
    from h2o.frame import H2OFrame
    from h2o.model.metrics import H2OBinomialModelMetrics
    
    h2o.init(nthreads=4, max_mem_size=12)
    best_model = h2o.load_model("C:\\Users\\axk220013\\Downloads\\artifacts_2\\best_model_path\\Grid_GBM_py_2_sid_924c_model_python_1683498529415_2743_model_3")


    with open('scaler.pkl', 'rb') as file:
        scaler = pickle.load(file)


    with open('target_encoder.pkl', 'rb') as file:
        target_encoder = pickle.load(file)



    data = pd.read_csv(dataset_file)
    test_data = data.drop(columns=['index']) #Removing the index column 

    # Cleaning and encoding test data
    # Convert strings styled as '$XXXX.XX' to float values in specific columns
    convertables = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
    for col in convertables:
        test_data[col] = test_data[col].str.replace('$', '')
        test_data[col] = test_data[col].str.replace(' ', '')
        test_data[col] = test_data[col].str.replace(',', '')
    test_data = test_data.astype({'DisbursementGross': 'float', 'BalanceGross': 'float', 'GrAppv': 'float', 'SBA_Appv': 'float'})

    # Remove records where RevLineCr not equals 'Y' or 'N' and LowDoc not equals 'Y' or 'N'
    test_data = test_data[(test_data['RevLineCr'] == 'Y') | (test_data['RevLineCr'] == 'N')]
    test_data = test_data[(test_data['LowDoc'] == 'Y') | (test_data['LowDoc'] == 'N')]

    # Replace 'Y' with 1 and 'N' with 0 in these two columns
    test_data['RevLineCr'] = test_data['RevLineCr'].replace({'Y': 1, 'N': 0})
    test_data['LowDoc'] = test_data['LowDoc'].replace({'Y': 1, 'N': 0})

    # Create a new column called 'Franchise_Exists' to determine if a franchise exists or not
    test_data.loc[(test_data['FranchiseCode'] <= 1), 'Franchise_Exists'] = 0
    test_data.loc[(test_data['FranchiseCode'] > 1), 'Franchise_Exists'] = 1
    test_data['Franchise_Exists'] = test_data['Franchise_Exists'].astype(int)
    test_data = test_data.drop('FranchiseCode', axis=1)

    # Convert 'NewExist' column to 0 and 1
    unique_values = test_data['NewExist'].unique()
    valid_values = [val for val in unique_values if val in [1, 2]]
    test_data = test_data[test_data['NewExist'].isin(valid_values)]
    test_data['NewExist'] = test_data['NewExist'].replace({1.: 0, 2.: 1}).astype(int)

    # Replace null values
    filler = {}
    for col in test_data.drop(columns=['MIS_Status']).columns:
        if test_data[col].dtype == 'object':
            filler[col] = "UnKnown Value"
        else:
            filler[col] = 0
    test_data.fillna(value=filler, inplace=True)

    # Replace NAICS values with first two digits to identify industry
    test_data['NAICS'] = test_data['NAICS'].apply(lambda x: str(x)[:2] if x != 0 else 0)
    test_data = test_data[test_data['NAICS'] != 0]

    # Create a feature for the percentage of loan approved by SBA with respect to total Gross approved
    test_data['percent_approved'] = test_data['SBA_Appv'] / test_data['GrAppv']

    # Scale selected columns in the dataset
    cols_to_scale = ['SBA_Appv', 'GrAppv', 'NoEmp', 'RetainedJob', 'CreateJob', 'LoanInd', 'percent_approved', 'DisbursementGross', 'BalanceGross']
    test_data[cols_to_scale] = scaler.transform(test_data[cols_to_scale])

    # Apply target encoding
    test_data = target_encoder.transform(test_data)

    for col in ["Zip", "City", "NAICS", 'Bank', 'BankState', 'State', 'UrbanRural', 'LowDoc', 'RevLineCr', 'Franchise_Exists', 'NewExist']:
        new_col_name = col + "_trg"
        test_data.rename(columns={col: new_col_name}, inplace=True)
   
    test_indices = test_data.index
    data = data[data.index.isin(test_indices)]

    test_data_h20 = h2o.H2OFrame(test_data)
    response = "MIS_Status"
    test_data_h20[response] = test_data_h20[response].asfactor()

    y_pred = best_model.predict(test_data_h20).as_data_frame()["predict"].values
    y_pred_proba = best_model.predict(test_data_h20).as_data_frame()["p1"].values
    y_pred_proba = np.column_stack((1 - y_pred_proba, y_pred_proba))
    threshold = 0.27
    y_pred = (y_pred_proba[:, 1] >= threshold).astype(np.int16)
    d = {"index": data["index"],
    "label": y_pred,
    "probability_0": y_pred_proba[:, 0],
    "probability_1": y_pred_proba[:, 1]}

    return pd.DataFrame(d)




In [30]:
scoring_function_project2("./data/SBA_loans_project_2.zip").head()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


--------------------------  --------------------------------
H2O_cluster_uptime:         11 hours 19 mins
H2O_cluster_timezone:       America/Chicago
H2O_data_parsing_timezone:  UTC
H2O_cluster_version:        3.38.0.4
H2O_cluster_version_age:    4 months and 2 days !!!
H2O_cluster_name:           H2O_from_python_axk220013_o7m6b5
H2O_cluster_total_nodes:    1
H2O_cluster_free_memory:    1.368 Gb
H2O_cluster_total_cores:    8
H2O_cluster_allowed_cores:  8
H2O_cluster_status:         locked, healthy
H2O_connection_url:         http://localhost:54321
H2O_connection_proxy:       {"http": null, "https": null}
H2O_internal_security:      False
Python_version:             3.8.10 final
--------------------------  --------------------------------

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


index  label  probability_0  probability_1
0      0      0       0.933249       0.066751
1      1      0       0.901645       0.098355
4      4      0       0.932651       0.067349
5      5      0       0.890940       0.109060
6      6      0       0.907380       0.092620